# Module 6

## Video 27: Multiple Vessels in a Movement
**Python for the Energy Industry**

Sometimes, a Cargo Movement will contain information on multiple vessels. There are two reasons this might happen:
- The cargo is loaded to, or unloaded from, a floating storage vessel
- A ship to ship transfer (STS) occurs

Let's find some example Cargo Movements where this is the case.

[Cargo Movements documentation](https://vortechsa.github.io/python-sdk/endpoints/cargo_movements/)

In [ ]:
from datetime import datetime

import vortexasdk as v

now = datetime.utcnow()

We start as before with a simple query:

In [ ]:
cm_query = v.CargoMovements().search(
    filter_activity="any_activity",
    filter_time_min=now,
    filter_time_max=now)

print(len(cm_query))

We can find Cargo Movements with STS events by getting those there multiple vessels are detailed:

In [ ]:
multi_vessel = [q for q in cm_query if len(q["vessels"]) > 1]
print(len(multi_vessel))

Let's take a look at the types of event happening in each of these:

In [ ]:
for cm in multi_vessel:
    event_types = [e["event_type"] for e in cm["events"]]
    print(event_types)

## STS Events

We'll look in more detail at those Cargo Movements with STS events.

In [ ]:
sts_movements = [cm for cm in multi_vessel if "cargo_sts_event" in [e["event_type"] for e in cm["events"]]]
print(len(sts_movements))

Let's take a look at the vessels in one of them:

In [ ]:
sts_movements[0]["vessels"][0]

In [ ]:
sts_movements[0]["vessels"][1]

 And also the STS event:

In [ ]:
sts_movements[0]["events"][0]

Let's view the data on movements with STS transfers in a DataFrame. The easiest way to do this is to convert our initial search into a DataFrame using the `to_df()` function, and then drop all movements with no STS event.

In [ ]:
required_columns = ["vessels.0.name","vessels.0.vessel_class","vessels.1.name","vessels.1.vessel_class","product.group.label","quantity",
                    "events.cargo_port_load_event.0.location.port.label","events.cargo_sts_event.0.location.sts_zone.label",
                    "events.cargo_port_unload_event.0.location.port.label"]

new_labels = ["vessel0","vessel0_class","vessel1","vessel1_class","product_group","quantity","loading_port","sts_location","unloading_port"]

relabel = dict(zip(required_columns,new_labels))

In [ ]:
cm_df = cm_query.to_df(columns=required_columns).rename(relabel,axis=1)

In [ ]:
cm_df_multi = cm_df.dropna(subset=["loading_port","sts_location","unloading_port"])

In [ ]:
cm_df_multi

### Exercise

Expand the range of the Cargo Movements - say to a ~2 week period. Can you find any movements involving 3 or more vessels? If so, isolate and have a look at these.